In [ ]:
# if you are using .DCM images
!pip install pydicom

AUGMENT YOUR DATA

In [ ]:
import pydicom
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def augment(dicom_path, output_path):
  dicom_data = pydicom.dcmread(dicom_path)
  original_filename, extension = os.path.splitext(os.path.basename(dicom_path))
  image = dicom_data.pixel_array
  image = np.expand_dims(image, axis=-1)
  datagen = ImageDataGenerator(
      vertical_flip=True
      # horizontal_flip=True
  )
  augmented_image = datagen.apply_transform(image, transform_parameters={
            # "flip_horizontal": True
            "flip_vertical": True
        })
  augmented_image = np.squeeze(augmented_image, axis=-1)
  dicom_data.PixelData = augmented_image.tobytes()
  new_filename = original_filename + "_molarv" + extension
  new_dicom_file_path = os.path.join(output_path, new_filename)
  dicom_data.save_as(new_dicom_file_path)
  print(f"Saved augmented image as {output_path}")
input_directory = "/path/to/images"
output_directory = "/path/to/augmented_images"
for filename in os.listdir(input_directory):
    if filename.endswith(".DCM"):
      dicom_path = os.path.join(input_directory, filename)
      augment(dicom_path, output_directory)

MOVE and ORGANIZE YOUR DATA IF NEEDED

In [ ]:
source_directory = '/path/to/source_directory'
output_directory = '/path/to/output_directory'
# List DICOM files in the source directory
dicom_files = [os.path.join(source_directory, filename) for filename in os.listdir(source_directory) if filename.endswith('.DCM')]
# c = 0
for dicom_file in dicom_files:
  ds = pydicom.dcmread(dicom_file)
  # c += 1
  file_name = os.path.splitext(os.path.basename(dicom_file))[0]
  # You can manipulate the DICOM data if needed.
  new_name = file_name + '.DCM'
  new_filename = os.path.join(output_directory, new_name)
  ds.save_as(new_filename)
# print(c)

LOAD YOUR DATA

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np
import cv2
from PIL import Image
path = "/path/to/your_images"
images = []
labels = []
for filename in os.listdir(path):
    ds = pydicom.read_file(os.path.join(path, filename))
    if 'premolar' in filename:
      labels.append(1)
    elif 'molar' in filename:
      labels.append(2)
    pixel_array = ds.pixel_array.astype(np.float32)
    image = Image.fromarray(pixel_array)
    size = (200 ,200)
    image = image.resize(size)
    # plt.imshow(image, cmap="gray")
    # plt.axis('off')
    # plt.show()
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = preprocess_input(image)
    images.append(image)

In [ ]:
images = np.array(images)
images = np.stack(images, axis=0)
print(len(labels))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the LabelEncoder to your class labels
label_encoder.fit(labels)

# Get the encoded labels
labels = label_encoder.transform(labels)

# Print the encoded labels
labels.shape

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=23, stratify=labels)

print("Train data shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation data shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
# print("Test data shape:", test_images.shape)
# print("Test labels shape:", test_labels.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l1_l2, l2, l1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(200, 200, 3))

model = keras.Sequential([
        base_model,
        layers.BatchNormalization(),
        layers.AveragePooling2D(pool_size=(2, 2), padding='same'),
        layers.Flatten(),
        layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='sigmoid')])

for layer in base_model.layers:
  layer.trainable = False
# for layer in base_model.layers[-10:]:
#     layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.00002), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=32, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/molar_premolar_model.h5')

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
# plt.plot(epochs, train_loss, 'b-', label='Training Loss')
# plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.optimizers import Adam
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# unfreeze layers
for layer in base_model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.000002), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, batch_size=32, epochs=100, validation_data=(val_images, val_labels), callbacks=[early_stopping])
model.save('/path/to/fine_tuned_molar_premolar_model.h5')

In [ ]:
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.plot(epochs, train_accuracy, 'b--', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r--', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.title('Training and Validation Metrics')
plt.legend()
plt.show()